In [6]:
from absl.testing import absltest
from absl.testing import parameterized
from brax import envs
from brax.training import apg

a, b, metrics = apg.train(
    environment_fn=envs.create_fn('inverted_double_pendulum'),
    episode_length=250,
    action_repeat=1,
    num_envs=16,
    learning_rate=3e-3,
    normalize_observations=True,
    log_frequency=200,
    truncation_length=None,
)

(128, 25)
FrozenDict({
    params: {
        hidden_0: {
            bias: (32,),
            kernel: (25, 32),
        },
        hidden_1: {
            bias: (32,),
            kernel: (32, 32),
        },
        hidden_2: {
            bias: (32,),
            kernel: (32, 32),
        },
        hidden_3: {
            bias: (32,),
            kernel: (32, 32),
        },
        hidden_4: {
            bias: (2,),
            kernel: (32, 2),
        },
    },
})


In [7]:
metrics

{'eval/episode_alive_bonus': DeviceArray(258.24777, dtype=float32),
 'eval/episode_dist_penalty': DeviceArray(15.533053, dtype=float32),
 'eval/episode_r_tot': DeviceArray(242.42384, dtype=float32),
 'eval/episode_reward': DeviceArray(242.42384, dtype=float32),
 'eval/episode_vel_penalty': DeviceArray(0.29085326, dtype=float32),
 'eval/completed_episodes': DeviceArray(1227., dtype=float32),
 'eval/avg_episode_length': DeviceArray(24.929096, dtype=float32),
 'speed/sps': 43497.53308875954,
 'speed/eval_sps': 275429.7182656376,
 'speed/training_walltime': 93.27130889892578,
 'speed/timestamp': 93.27130889892578,
 'train/grad_norm': DeviceArray(0.5268832, dtype=float32),
 'train/params_norm': DeviceArray(12.151604, dtype=float32)}

((DeviceArray(80000., dtype=float32),
  DeviceArray(0., dtype=float32),
  DeviceArray(0., dtype=float32)),
 FrozenDict({
     params: {
         hidden_0: {
             bias: DeviceArray([-0.0109704 ,  0.05863323,  0.03235909,  0.10621556,
                           0.0195198 ,  0.06637241,  0.00995906,  0.07235441,
                          -0.05737618, -0.01933287, -0.08531769,  0.0161806 ,
                           0.05896898,  0.0083474 , -0.02627146, -0.06677905,
                          -0.09020948,  0.02276647, -0.04713441, -0.0252438 ,
                          -0.04737217,  0.01300221, -0.05602237, -0.00527998,
                          -0.02130723,  0.02284122, -0.00031477,  0.04149071,
                           0.03073505, -0.01461188,  0.01178026, -0.03072847],            dtype=float32),
             kernel: DeviceArray([[ 2.21062958e-01, -5.73815823e-01, -9.71614718e-02,
                           -3.68516833e-01,  7.42056444e-02,  8.76443908e-02,
                     